# SMA Crossover Strategy
Author: TraderPy

Youtube: https://www.youtube.com/channel/UC9xYCyyR_G3LIuJ_LlTiEVQ/featured

Website: https://traderpy.com/

## Disclaimer
Trading the financial markets imposes a risk of financial loss. TraderPy is not responsible for any financial losses that viewers suffer. Content is educational only and does not serve as financial advice. Information or material is provided ‘as is’ without any warranty. 

Past trading results do not indicate future performance. Strategies that worked in the past may not reflect the same results in the future.

In [60]:
import MetaTrader5 as mt5
import pandas as pd
import plotly.express as px
import numpy as np

In [61]:
# connect to MetaTrader5 as mt5
mt5.initialize()

True

In [62]:
# settings
symbol = 'EURUSD'
timeframe = mt5.TIMEFRAME_D1
start_pos = 0
num_bars = 1000

fast_sma_period = 10
slow_sma_period = 100

In [63]:
# Requesting historical data
bars = mt5.copy_rates_from_pos(symbol, timeframe, start_pos, num_bars)

df = pd.DataFrame(bars)[['time', 'open', 'high', 'low', 'close', 'spread']]
df['time'] = pd.to_datetime(df['time'], unit='s')

df['fast_sma'] = df['close'].rolling(fast_sma_period).mean()
df['slow_sma'] = df['close'].rolling(slow_sma_period).mean()

# finding crossovers
df['prev_fast_sma'] = df['fast_sma'].shift(1)

df.dropna(inplace=True)
df

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma
99,2020-07-14,1.13431,1.14077,1.13242,1.13993,2,1.129689,1.105013,1.128032
100,2020-07-15,1.13990,1.14510,1.13902,1.14114,2,1.131296,1.105544,1.129689
101,2020-07-16,1.14113,1.14411,1.13695,1.13834,2,1.132742,1.105928,1.131296
102,2020-07-17,1.13832,1.14429,1.13745,1.14258,2,1.134550,1.106325,1.132742
103,2020-07-20,1.14107,1.14668,1.14013,1.14472,2,1.135937,1.106639,1.134550
...,...,...,...,...,...,...,...,...,...
995,2023-12-22,1.10036,1.10395,1.09929,1.10115,16,1.091815,1.075375,1.089316
996,2023-12-26,1.10153,1.10441,1.10078,1.10407,16,1.094592,1.075415,1.091815
997,2023-12-27,1.10402,1.11217,1.10276,1.11057,15,1.097708,1.075566,1.094592
998,2023-12-28,1.10998,1.11386,1.10542,1.10611,15,1.099579,1.075654,1.097708


In [64]:
# Visualize Close Price
px.line(df, x='time', y='close')

In [65]:
def find_crossover(fast_sma, prev_fast_sma, slow_sma):
    
    if fast_sma > slow_sma and prev_fast_sma < slow_sma:
        return 'bullish crossover'
    elif fast_sma < slow_sma and prev_fast_sma > slow_sma:
        return 'bearish crossover'
    
    return None


df['crossover'] = np.vectorize(find_crossover)(df['fast_sma'], df['prev_fast_sma'], df['slow_sma'])

# df.dropna(inplace=True)
# signal = df[df['crossover'] == 'bullish crossover'].copy() # => Original
signal = df[(df['crossover'] == 'bullish crossover') | (df['crossover'] == 'bearish crossover')].copy()

signal

,time,open,high,low,close,spread,fast_sma,slow_sma,prev_fast_sma,crossover
267,2021-03-09,1.18447,1.19147,1.18346,1.18992,16,1.202320,1.203199,1.204812,bearish crossover
304,2021-04-29,1.21230,1.21490,1.21009,1.21172,20,1.206023,1.205323,1.204496,bullish crossover
342,2021-06-22,1.19159,1.19515,1.18800,1.19399,16,1.203132,1.203200,1.205451,bearish crossover
343,2021-06-23,1.19355,1.19693,1.19106,1.19247,16,1.200612,1.203090,1.203132,bearish crossover
706,2022-11-14,1.03334,1.03578,1.02704,1.03250,18,1.003800,1.002822,0.999376,bullish crossover
844,2023-05-25,1.07504,1.07557,1.07065,1.07242,10,1.080501,1.081419,1.082415,bearish crossover
861,2023-06-19,1.09359,1.09454,1.09063,1.09216,10,1.081020,1.080652,1.078920,bullish crossover
905,2023-08-18,1.08665,1.08929,1.08441,1.08712,10,1.092793,1.093049,1.094180,bearish crossover
972,2023-11-21,1.09384,1.09641,1.08990,1.09111,10,1.080956,1.079108,1.078828,bullish crossover


In [66]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
fig.show()

In [67]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
        return True
        
# logic
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.crossover == 'bearish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'sell', self.volume, 0, 0))
            
            if data.crossover == 'bullish crossover':
                for position in self.positions:
                    if position.status == 'open':
                        position.close_position(data.time, data.close)
                self.add_position(Position(data.time, data.close, 'buy', self.volume, 0, 0))
        
        return self.get_positions_df()

In [68]:
# Lot Size:
# Standard => Volume = 100,000 => Lot = 1.00 => pip Value = 10.00 USD
# Mini Lot => Volume = 10,000 => Lot = 0.10 => pip Value = 1.00 USD
# Micro Lot => Volume = 1,000 => Lot = 0.01 => pip Value = 0.10 USD
# Nano Lot => Volume = 100 => Lot = 0.0001 => pip Value = 10 USD

starting_balance = 1000 
standard_contract = 100000
initial_volume = 10000 # pip = 1 USD
initial_lot = initial_volume / standard_contract
volume = initial_lot * standard_contract 


In [69]:
sma_crossover_strategy = Strategy(df, starting_balance, volume)
result = sma_crossover_strategy.run()

result

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl
0,2021-03-09,1.18992,sell,10000.0,0,0,2021-04-29,1.21172,-218.0,closed,782.0
1,2021-04-29,1.21172,buy,10000.0,0,0,2021-06-22,1.19399,-177.3,closed,604.7
2,2021-06-22,1.19399,sell,10000.0,0,0,2021-06-23,1.19247,15.2,closed,619.9
3,2021-06-23,1.19247,sell,10000.0,0,0,2022-11-14,1.03250,1599.7,closed,2219.6
4,2022-11-14,1.03250,buy,10000.0,0,0,2023-05-25,1.07242,399.2,closed,2618.8
5,2023-05-25,1.07242,sell,10000.0,0,0,2023-06-19,1.09216,-197.4,closed,2421.4
6,2023-06-19,1.09216,buy,10000.0,0,0,2023-08-18,1.08712,-50.4,closed,2371.0
7,2023-08-18,1.08712,sell,10000.0,0,0,2023-11-21,1.09111,-39.9,closed,2331.1
8,2023-11-21,1.09111,buy,10000.0,0,0,NaT,NaN,NaN,open,NaN


In [70]:
#  Get Columns Titles to check Existing Columns
titles = result.columns

for title in titles:
 print(title)


open_datetime
open_price
order_type
volume
sl
tp
close_datetime
close_price
profit
status
pnl


In [71]:
#  Get total profit
total_profit = result['profit'].sum()

total_profit

1331.1000000000006

In [72]:
px.line(result, x='close_datetime', y='pnl')

In [73]:
# visualize close price
fig = px.line(df, x='time', y=['close', 'fast_sma', 'slow_sma'])

for i, row in signal.iterrows():
    fig.add_vline(x=row.time)
    
for i, row in result[result['status'] == 'closed'].iterrows():
    
    if row.profit > 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Green",width=3)
                     )
                      
    elif row.profit < 0:
        fig.add_shape(type="line",
            x0=row.open_datetime, y0=row.open_price, x1=row.close_datetime, y1=row.close_price,
            line=dict(color="Red",width=3)
                      )

    
fig.show()

In [74]:

def create_profit_type(profit):
    # Finding Signal
    if profit < 0:
        return 'loss'
    elif profit > 0:
        return 'profit'
    
result['profit_type'] = np.vectorize(create_profit_type)(result['profit'])

result.dropna(inplace=True)

display(result)



# i = -1
# last= result['profit'].iloc[i]

# while last > 0:
#     i -= 1
#     display(i)
#     last= result['profit'].iloc[i]
#     print(last)


d:\Private\PythonProjects\python_finance\lib\site-packages\numpy\lib\function_base.py:2246: RuntimeWarning:

invalid value encountered in create_profit_type (vectorized)



,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,profit_type
0,2021-03-09,1.18992,sell,10000.0,0,0,2021-04-29,1.21172,-218.0,closed,782.0,loss
1,2021-04-29,1.21172,buy,10000.0,0,0,2021-06-22,1.19399,-177.3,closed,604.7,loss
2,2021-06-22,1.19399,sell,10000.0,0,0,2021-06-23,1.19247,15.2,closed,619.9,profit
3,2021-06-23,1.19247,sell,10000.0,0,0,2022-11-14,1.03250,1599.7,closed,2219.6,profit
4,2022-11-14,1.03250,buy,10000.0,0,0,2023-05-25,1.07242,399.2,closed,2618.8,profit
5,2023-05-25,1.07242,sell,10000.0,0,0,2023-06-19,1.09216,-197.4,closed,2421.4,loss
6,2023-06-19,1.09216,buy,10000.0,0,0,2023-08-18,1.08712,-50.4,closed,2371.0,loss
7,2023-08-18,1.08712,sell,10000.0,0,0,2023-11-21,1.09111,-39.9,closed,2331.1,loss


In [75]:
result['grand_profit'] = (result.groupby(result['profit_type'].ne(result['profit_type'].shift()).cumsum()).cumcount())

display(result)

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,profit_type,grand_profit
0,2021-03-09,1.18992,sell,10000.0,0,0,2021-04-29,1.21172,-218.0,closed,782.0,loss,0
1,2021-04-29,1.21172,buy,10000.0,0,0,2021-06-22,1.19399,-177.3,closed,604.7,loss,1
2,2021-06-22,1.19399,sell,10000.0,0,0,2021-06-23,1.19247,15.2,closed,619.9,profit,0
3,2021-06-23,1.19247,sell,10000.0,0,0,2022-11-14,1.03250,1599.7,closed,2219.6,profit,1
4,2022-11-14,1.03250,buy,10000.0,0,0,2023-05-25,1.07242,399.2,closed,2618.8,profit,2
5,2023-05-25,1.07242,sell,10000.0,0,0,2023-06-19,1.09216,-197.4,closed,2421.4,loss,0
6,2023-06-19,1.09216,buy,10000.0,0,0,2023-08-18,1.08712,-50.4,closed,2371.0,loss,1
7,2023-08-18,1.08712,sell,10000.0,0,0,2023-11-21,1.09111,-39.9,closed,2331.1,loss,2


In [76]:
# (result.profit_type != result.profit_type.shift()).cumsum()

# display(result)

In [77]:
last= result['profit_type'].iloc[-1]

new_result = pd.DataFrame()

if last == 'loss':
    last_index= result['grand_profit'].iloc[-1] + 1
    new_result = result.tail(last_index)

display(new_result)

,open_datetime,open_price,order_type,volume,sl,tp,close_datetime,close_price,profit,status,pnl,profit_type,grand_profit
5,2023-05-25,1.07242,sell,10000.0,0,0,2023-06-19,1.09216,-197.4,closed,2421.4,loss,0
6,2023-06-19,1.09216,buy,10000.0,0,0,2023-08-18,1.08712,-50.4,closed,2371.0,loss,1
7,2023-08-18,1.08712,sell,10000.0,0,0,2023-11-21,1.09111,-39.9,closed,2331.1,loss,2


In [78]:
last_loss = new_result['profit'].sum()

last_loss

-287.6999999999996